## Машинное обучение в бизнесе
### Урок 9. Интеграция. Итоговый проект

Реализация REST API на базе Flask (пример - https://github.com/fimochka-sudo/GB_docker_flask_example)

1. Выбрать датасет, сделать pipeline (преобразования и модель) для удобного вызова из кода сервиса и сохранить его на диск.
2. Выполнить разработку представляется целесообразным в PyCharm.
3. Для выполнения проекта потребуется файл requirements.txt с пакетами; за основу можно взять аналогичный файл из проекта по ссылке (см. выше); установку перечисленных в нём пакетов можно выполнить из терминала в PyCharm по команде pip install -r requirements.txt (находясь при этом в корне проекта).
4. Итоговый проект должен содержать в корневом каталоге:
   - каталог app/models/ (предобученная модель-пайплайн либо код обучения модели-пайплайна);
   - файл app/run_server.py (основной код flask-приложения);
   - файл requirements.txt (список использованных пакетов);
   - README.md (описание проекта, данных к нему, процедуры запуска и т.п.);
   - файл Dockerfile;
   - файл docker-entrypoint.sh.
5. Проект должен быть размещён в репозитории на GitHub (пример работы с репозиторием - https://guides.github.com/activities/hello-world/);
6. (<b>Опционально</b>): front-end сервис, который умеет принимать от пользователя введеные данные и использовать API проекта.

**Полезные ссылки:**
1. датасеты (для полета мысли): https://www.kaggle.com/datasets
2. конкурс Сбербанка по недвижимости с примерами ноутбуков, реализующих разные подходы (можно этот набор данных также взять и обучить модель предсказывать стоимость жилья - неплохой сервис может получиться) - https://www.kaggle.com/c/sberbank-russian-housing-market/data
3. минималистичный пример связки keras/flask для определения класса картинки - https://blog.keras.io/building-a-simple-keras-deep-learning-rest-api.html
4. неплохой пример связки docker/flask (помимо разобранного на занятии) - https://cloud.croc.ru/blog/byt-v-teme/flask-prilozheniya-v-docker/
5. https://www.digitalocean.com/community/tutorials/how-to-build-and-deploy-a-flask-application-using-docker-on-ubuntu-18-04

### Также допускается стандартная версия реализации REST API на базе Flask в Google Collab или Jupyter Notebook

1. Выбрать датасет, сделать pipeline (преобразования и модель) для удобного вызова из кода сервиса и сохранить его на диск.
2. Реализовать ноутбук с сервером
3. Реализовать ноутбук с клиентом

### 1. Создание, обучение и сохранение модели с обучающей и тестовой выборками
Dataset и Pipeline с сохранением на диске в расчёте на последующий вызов из кода сервиса

In [1]:
# Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
# Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
import dill
# Data normalizing
from sklearn.preprocessing import StandardScaler
# Imputer
from sklearn.impute import SimpleImputer
# Working with a text
from sklearn.feature_extraction.text import TfidfVectorizer
# Auxiliary
import numpy as np

In [2]:
df = pd.read_csv("fake_job_postings.csv")
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [4]:
print(df.shape)
df['fraudulent'].value_counts()

(17880, 18)


0    17014
1      866
Name: fraudulent, dtype: int64

Разделение датасета на обучающую и тестовую выборки и их сохранение для последующего прямого вызова

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, df['fraudulent'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

Создание Pipeline с простейшим классификатором и его сохранение для последующего использования

In [6]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [7]:
features = ['description', 'company_profile', 'benefits']
target = 'fraudulent'

In [32]:
# Функциональность обработки признаков

description = Pipeline([
                ('imputer', TextImputer('description', '')),
                ('selector', ColumnSelector(key='description')),
                ('tfidf', TfidfVectorizer())])

company_profile = Pipeline([
                ('imputer', TextImputer('company_profile', '')),
                ('selector', ColumnSelector(key='company_profile')),
                ('tfidf', TfidfVectorizer())])

benefits = Pipeline([
                ('imputer', TextImputer('benefits', '')),
                ('selector', ColumnSelector(key='benefits')),
                ('tfidf', TfidfVectorizer())])

feats = FeatureUnion([('description', description),
                      ('company_profile', company_profile),
                      ('benefits', benefits)])

# max_df=0.9, min_df=10 - было в параметрах векторайзера

Pipeline: создание, обучение, просмотр, сохранение

In [33]:
%%time

pipeline = Pipeline([('features', feats), ('classifier', LogisticRegression()), ])

pipeline.fit(X_train, y_train)

Wall time: 13 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('description',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='description',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='description')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('company_profile',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='company_profile',
                                                     

In [34]:
# Просмотр

pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('description',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='description',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='description')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('company_profile',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='company_profile',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='company_profile')),
                                                  ('tfi

In [35]:
# Сохранение модели для последующих вызовов

with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

### 2. Проверка работоспособности и качества модели (pipeline)
Загрузка и проверка модели на отложенной (тестовой) выборке без запуска API

In [36]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [37]:
print(y_test.shape)

(5901, 1)


In [38]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [39]:
print(X_test.shape)
X_test.head(3)

(5901, 18)


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,4709,Python Engineer,"GB, , London",NaN,NaN,NaN,Stylect is a dynamic startup that helps helps ...,We don’t care where you studied or what your G...,We are negotiable on salary and there is the p...,0,1,0,Full-time,Entry level,Unspecified,Apparel & Fashion,Information Technology,0
1,11080,Entry Level Sales,"US, OH, Cincinnati",NaN,55000-75000,NaN,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...,1,0,0,Full-time,Entry level,High School or equivalent,Financial Services,Sales,0
2,12358,Agile Project Manager,"US, NY, New York",NaN,NaN,ustwo offers you the opportunity to be yoursel...,"At ustwo™ you get to be yourself, whilst deliv...",Skills• Experience interfacing directly with c...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


Пробное предсказание и сохранение его результатов

In [40]:
predictions = pipeline.predict_proba(X_test)[:, 1]
pd.DataFrame({'preds': predictions}).to_csv("test_predictions.csv", index=None)

In [41]:
print(predictions.shape)

(5901,)


Расчёт и просмотр метрик качества

In [42]:
roc_auc_score(y_score=predictions, y_true=y_test)

0.988583269950173

In [43]:
predictions[:10]

array([0.02602203, 0.04317015, 0.00370601, 0.00112958, 0.00151454,
       0.00213981, 0.00256837, 0.00373913, 0.00069803, 0.0122069 ])

In [44]:
precision, recall, thresholds = precision_recall_curve(y_test, predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold = {thresholds[ix]}, F-Score = {fscore[ix]:.3f}, Precision = {precision[ix]:.3f}, Recall = {recall[ix]:.3f}')

Best Threshold = 0.20397799406948391, F-Score = 0.852, Precision = 0.900, Recall = 0.809


### 3. Проверка работоспособности API модели, или допустимых запросов к веб-серверу
1. Создание docker-образа и запуск контейнера по инструкциям на https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Непосредственно запуск кода для проверки работоспособности API

Клонирование репозитория и создание образа

In [45]:
!git clone https://github.com/fimochka-sudo/GB_docker_flask_example.git
!cd GB_docker_flask_example
!docker build -t fimochka/gb_docker_flask_example .

fatal: destination path 'GB_docker_flask_example' already exists and is not an empty directory.
"docker" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [46]:
from urllib import request, parse
import json

In [47]:
def get_prediction(x):
    description, company_profile, benefits = x
    body = {'description': description, 
            'company_profile': company_profile,
            'benefits': benefits
            } 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

***Пока не разобрались с docker (а собрать дистрибутив по аналогии с программами типа CreateInstall из 1990-2000-х годов можно), повторим материал урока - подход с веб-сервером на Flask и с 2-мя вкладками (как на уроке): сервером - этот блокнот - и клиентом - переход на новую вкладку по ссылке http://127.0.0.1:5000/ и подачей запросов после слэша (по аналогии с запуском модели в collab на уроке, всё работает)***

In [48]:
from flask import Flask, request, jsonify

В имени, коль из этого модуля непосредственно запускается код, будет '__main__'. Воспроизведём и запустим сервер

In [49]:
__name__

'__main__'

In [50]:
app = Flask(__name__)

@app.route("/a")
def hello_a():
    return "Hello, World!"

@app.route("/b")
def hello_b():
    return "Hello, GeekBrains!"

Проверено по аналогии с *Collab*, как на уроке - работает, меняя приветствие в зависимости от обращения http://127.0.0.1:5000/a или http://127.0.0.1:5000/b

In [51]:
#if __name__ == '__main__':
#    app.run()

### 4. Сервис для обработки запросов к модели с помощью веб-сервера *Flask*
Загрузка обученной модели и данных (далее этот блокнот будет работать как серверный, а клиентская часть будет написана в отдельном блокноте)

In [56]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [57]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Почти рабочий сервер (как сказано и написано, в силу отладочного предназначения Flask), откликающийся на запросы "по существу" к модели 

In [62]:
# Разница в обработчиках - Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])            # сразу при подключении клиента (слэш будет последним символом в строке) выдать приветствие
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])    # выполнение предсказания
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    description, company_profile, benefits = "", "", ""
    request_json = request.get_json()
    
    if request_json["description"]:
        description = request_json['description']
    
    if request_json["company_profile"]:
        company_profile = request_json['company_profile']
                
    if request_json["benefits"]:
        benefits = request_json['benefits']
    
    print(description)  
    preds = model.predict_proba(pd.DataFrame({"description": [description],
                                              "company_profile": [company_profile],
                                              "benefits": [benefits]}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)

И его запуск/останов

In [64]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2023 22:22:01] "POST //predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK


127.0.0.1 - - [17/Jul/2023 22:22:19] "POST //predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK


...

Benefits with TPC: Pay on bench,Opportunity to work with 500 fortune Companies Training on New Technologies coming up in these areas. Medical and Health nsurance GC processingJob Description: Experience required: Good in  any coding - C/C++/.net/Java.Job Type: Full Time with The Principal Consulting.Job Location: Texas or New Jersey (Should be open to relocate and travelling at different Client locations.). Job Skills2-6 years experience web development on any platform like C/C++/.net/Java.Strong on coding, programming and OOPS.Open to learn new Mobile Technologies like ios - Objective C / Android / Xcode / SDK / Cocoa etc.Previous experience with .NET/ Java/ C++/ C is desirable.Strong experience detecting / correcting memory usage issuesExperience in agile development is an advantageshould be a good learner. 
OK
The Head of Communications will be responsible for executing a media relations strategy that results in strong journalist relationships, positive news coverage and compelling 

127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -


The Support Engineer will act as an escalation point for complex technical system incident resolution and will be responsible for ensuring application availability to end users in accordance with agreed Service Level Agreements.Participates as a team member in the implementation and support of applications in a production environmentPerforms support for one or more applications and releases of varying levels of complexity (ranging from medium to high)Responsible for ensuring each reported problem is resolved in a timely manner, tracked according to standards and escalated as appropriateParticipates in the implementation of new releases into productionParticipates in project team activities prior to release and contributes to documentation requirements consistent with methodologyResponsible for solving medium to high complexity support problemsResponsible for developing and maintaining information for a knowledge baseTracks feature and bug requests for prioritizationCommunicates problem

127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2023 22:52:16] "POST //predict HTTP/1.1" 200 -


The right candidate will be an integral part of our talented team, supporting our continued growth.Key responsibilities include:Production Print operations – maintains a working knowledge of all production print and copy equipment on site.Operates various Production scale printers and organizes the daily print flow.Assists technical support and onsite techs with trouble shooting and basic maintenance.Perform any and all duties as assigned by management to include; mail services, reprographics services, fax services, and messenger services.Establish operating procedures and quality standards.Responsible for all aspects of equipment.Remain informed and adhere to all established safety and security procedures.Perform necessary adjustments as outlined in the operators’ manual for set equipment.Follow start-up, shutdown and clean-up procedures for equipment.Monitor operations and make mechanical adjustments as needed, to ensure proper quality output.Create excel spreadsheets, mail merge pro